## Задача 1.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [86]:
from scipy.stats import mannwhitneyu, shapiro, norm, t, kstest, shapiro

import numpy as np

import warnings

from statsmodels.stats import proportion

import matplotlib.pyplot as plt

import seaborn as sns

import pandas as pd

warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')

### Импорт данных

In [68]:
data = pd.read_excel('AB_Test_Results.xlsx')

data.head(10)

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
8,1574,gate_40,108,1,1
9,1587,gate_40,153,1,0


In [69]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [70]:
data.shape

(90189, 5)

### Проверка данных

In [71]:
data.isna().sum()

userid            0
version           0
sum_gamerounds    0
retention_1       0
retention_7       0
dtype: int64

In [72]:
data[data['version'] == 'gate_30'].describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.470000e+04,44700.000000,44700.000000,44700.000000
mean,4.987564e+06,52.456264,0.448188,0.190201
std,2.881026e+06,256.716423,0.497314,0.392464
min,1.160000e+02,0.000000,0.000000,0.000000
25%,2.505469e+06,5.000000,0.000000,0.000000
50%,4.983631e+06,17.000000,0.000000,0.000000
75%,7.481497e+06,50.000000,1.000000,0.000000
max,9.999710e+06,49854.000000,1.000000,1.000000


In [73]:
data[data['version'] == 'gate_40'].describe()

,userid,sum_gamerounds,retention_1,retention_7
count,4.548900e+04,45489.000000,45489.000000,45489.000000
mean,5.009073e+06,51.298776,0.442283,0.182000
std,2.885496e+06,103.294416,0.496663,0.385849
min,3.770000e+02,0.000000,0.000000,0.000000
25%,2.517171e+06,5.000000,0.000000,0.000000
50%,5.007329e+06,16.000000,0.000000,0.000000
75%,7.510762e+06,52.000000,1.000000,0.000000
max,9.999861e+06,2640.000000,1.000000,1.000000


In [74]:
data['userid'].nunique() == data['userid'].count()

True

Дублирования нет, данные корректны.

### Анализ данных

Распределение по группам.

In [75]:
data.groupby('version')[['userid']].nunique()

,userid
version,
gate_30,44700
gate_40,45489


Распределение по группам вполне корректное, группы практически одинаковы по численности.

In [76]:
data[data["sum_gamerounds"] == 0]["userid"].count()

3994

In [77]:
data[(data["sum_gamerounds"] == 0) & (data["version"] == 'gate_30')]["userid"].count()

1937

In [78]:
data[(data["sum_gamerounds"] == 0) & (data["version"] == 'gate_40')]["userid"].count()

2057

3994 игрока не играли в игру после скачивания вообще. Однако их число в каждой из групп почти одинаковое, значит игнорируем не играющих пользователей.

In [79]:
print(kstest(data['sum_gamerounds'], 'norm'))

KstestResult(statistic=0.8715607041848303, pvalue=0.0)


Проверяем на нормальность по Колмагорову-Смирнову и видим, что pvalue имеет значение близкое к нулевому, значит распределение нормальным считать не следует!

Делим пользователей на контрольную и тестовую группы:

In [80]:
data_control = data[data['version'] == 'gate_30']

data_test = data[data['version'] == 'gate_40']

1. Анализ результатов первого дня - Retention 1 day.

In [81]:
data.groupby('version')['retention_1'].mean()

version
gate_30    0.448188
gate_40    0.442283
Name: retention_1, dtype: float64

In [82]:
mannwhitneyu(x = data_control['sum_gamerounds'].values, y = data_test['sum_gamerounds'].values)

MannwhitneyuResult(statistic=1024331250.5, pvalue=0.05020880772044255)

Значение pvalue мало, практически равно по значению alpha, требуется дополнительная проверка

In [90]:
k1 = data_test['retention_1'].sum()
k2 = data_control['retention_1'].sum()

n1 = data_test.shape[0]
n2 = data_control.shape[0]

z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), np.array([n1, n2]))

print(f'Результаты:','z_score =', z_score, 'pvalue =', z_pvalue)

Результаты: z_score = -1.7840862247974725 pvalue = 0.07440965529691913


Принимаем нулевую гипотезу, так как pvalue > alpha. Статистических различий не видим межлу выборками в результате первого дня.

2. Анализ результатов за неделю - Retention 7 days.

In [13]:
data.groupby('version')['retention_7'].mean()

version
gate_30    0.190201
gate_40    0.182000
Name: retention_7, dtype: float64

In [91]:
k1 = data_test['retention_7'].sum()
k2 = data_control['retention_7'].sum()

n1 = data_test.shape[0]
n2 = data_control.shape[0]

z_score, z_pvalue = proportion.proportions_ztest(np.array([k1, k2]), np.array([n1, n2]))

print(f'Результаты:','z_score =', z_score, 'pvalue =', z_pvalue)

Результаты: z_score = -3.164358912748191 pvalue = 0.001554249975614329


Отвеграем нулевую гипотезу, так как pvalue < alpha. По истечении семи дней статистически значимое различие между выборками есть.

### Вывод: на основе представленных данных видно, что:
1. На основе данных за первый день видно, что статистически значимых различий между контрольной группой и тестовой группой нет, следовательно считаем этот результат промежуточным, не оринетируемся на эту выборку при формулировании рекомендаций.
2. Через неделю статистически значимая разница между выборками появляется, при этом игроки контрольной группы играют больше.
3. В результате ретроспективного анализа за 1 и за 7 дней, видим, что представленное изменение в игре не сказывается на начальном этапе тестирования, требуется хотя бы неделя, чтобы оценить результат. Причем, по истечению недели, игроки проявили больше интереса к игре в контрольной группе!
4. Не рекомендуется внедрять предложенное обновление, так как тренд показывает негативную динамику, которая подтверждена статистически.